## SENTIMENT ANALYSIS - PANDAS + COMPREHEND

In [1]:
## IMPORTS
from google_play_scraper import app
from google_play_scraper import Sort, reviews
import pandas as pd
import numpy as np
import boto3

In [12]:
## SCRAPE REVIEWS
result, continuation_token = reviews(
    'posteitaliane.posteapp.apppostepay',
    lang='it',
    country='it',
    sort=Sort.NEWEST,
    count=3)

In [13]:
## AGGIUNGO RISULTATO SCRAPING REVIEWS A DF
df = pd.DataFrame(np.array(result),columns=['review'])
df = df.join(pd.DataFrame(df.pop('review').tolist()))

In [14]:
## PER IL NOSTRO USE CASE SELEZIONO SOLO COLONNE 'content' e 'score'
df = df[['content','score']]
df.head()

,content,score
0,Si inventa fantomatici problemi tecnici divers...,1
1,"L'app di per sé è molto buona, comoda e utile ...",3
2,App molto poco intuitiva per un utente che non...,1


In [15]:
## COMPREHEND SENTIMENT ANALYSIS
client = boto3.client('comprehend')

def sentiment_analysis(test):
    response = client.detect_sentiment(
        Text=test,
        LanguageCode='it'
    )
    return response
    
sentiments = []

for review in df['content']:
    sent = sentiment_analysis(review)
    sentiments.append(sent['Sentiment'])
    
df['sentiment'] = sentiments

df.head()

,content,score,sentiment
0,Si inventa fantomatici problemi tecnici divers...,1,NEGATIVE
1,"L'app di per sé è molto buona, comoda e utile ...",3,MIXED
2,App molto poco intuitiva per un utente che non...,1,NEGATIVE


In [22]:
## CONTO REVIEWS IN BASE A SENTIMENT
numpositive = len(df.loc[df['sentiment'] == 'POSITIVE'])
numnegative = len(df.loc[df['sentiment'] == 'NEGATIVE'])
numneutral = len(df.loc[df['sentiment'] == 'MIXED'])
total = numpositive + numnegative + numneutral
print("Recensioni totali: "+str(total))
print("Ci sono "+str(numpositive)+" recensioni con sentiment positivo - Percentuale: "+str(numpositive/total)+"%")
print("Ci sono "+str(numnegative)+" recensioni con sentiment negativo - Percentuale: "+str(numnegative/total)+"%")
print("Ci sono "+str(numneutral)+" recensioni con sentiment neutrale - Percentuale: "+str(numneutral/total)+"%")

Recensioni totali: 3
Ci sono 0 recensioni con sentiment positivo - Percentuale: 0.0%
Ci sono 2 recensioni con sentiment negativo - Percentuale: 0.6666666666666666%
Ci sono 1 recensioni con sentiment neutrale - Percentuale: 0.3333333333333333%


In [23]:
## SALVO FILE EXCEL (SENZA COLONNA INDEX)
df.to_excel('result.xlsx',index=False)